In [7]:
import tkinter as tk
from tkinter import filedialog
import cv2
from PIL import Image, ImageTk
import numpy as np
from keras.models import load_model
import os

current_dir = os.getcwd()
saved_models_dir = os.path.join(current_dir, '..', 'Saved models')

sleep_awake_model_path = os.path.abspath(os.path.join(saved_models_dir, 'driver_detection_50_epochs.model'))
age_detection_model_path = os.path.abspath(os.path.join(saved_models_dir, 'vgg_model_age_detection.h5'))

sleep_awake_model = load_model(sleep_awake_model_path)
age_detection_model = load_model(age_detection_model_path)

age_labels = ['6-20', '25-30', '42-48', '60-98']

def preprocess_frame(frame):
    # Resize frame for sleep-awake model
    resized_frame = cv2.resize(frame, (64, 64))
    input_frame = np.expand_dims(resized_frame, axis=0) / 255.0
    return input_frame

def detect_sleep_awake(frame):
    input_frame_sleep_awake = preprocess_frame(frame)
    sleep_awake_prediction = sleep_awake_model.predict(input_frame_sleep_awake)
    return sleep_awake_prediction[0][0] > 0.5

def predict_age(frame):
    resized_frame_age = cv2.resize(frame, (224, 224))
    input_frame_age = np.expand_dims(resized_frame_age, axis=0)
    age_prediction = age_detection_model.predict(input_frame_age)
    predicted_age = age_labels[np.argmax(age_prediction)]
    return predicted_age

def integrate_models(frame):
    is_sleeping = detect_sleep_awake(frame)

    # Overlay text on the frame
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.6  # Reduce the font size
    color = (0, 255, 0)
    thickness = 2

    if is_sleeping:
        driver_status_text = "Driver is sleeping"
        predicted_age = predict_age(frame)
        age_group_text = f"Age group: {predicted_age}"
        cv2.putText(frame, age_group_text, (20, 100), font, fontScale, color, thickness, cv2.LINE_AA)
    else:
        driver_status_text = "Driver is awake"

    # Draw text for driver status
    cv2.putText(frame, driver_status_text, (20, 50), font, fontScale, color, thickness, cv2.LINE_AA)

    # Resize the output image to accommodate the text
    frame = cv2.resize(frame, (800, 600))

    return frame

# Define GUI
class DriverDetectionApp:
    def __init__(self, window):
        self.window = window
        self.window.title("Driver Detection App")

        # Image frame
        self.frame = tk.Frame(self.window, padx=10, pady=10)
        self.frame.pack()

        # Load image button
        self.load_image_button = tk.Button(self.frame, text="Load Image", command=self.load_image)
        self.load_image_button.pack(side=tk.LEFT)

        # Detect button
        self.detect_button = tk.Button(self.frame, text="Detect", command=self.detect)
        self.detect_button.pack(side=tk.LEFT)

        # Output label
        self.output_label = tk.Label(self.window, text="")
        self.output_label.pack()

        # Image display
        self.image_label = tk.Label(self.window)
        self.image_label.pack()

        self.image_path = ""

    def load_image(self):
        self.image_path = filedialog.askopenfilename(initialdir="./", title="Select Image File",
                                                     filetypes=(("JPEG files", "*.jpg"), ("PNG files", "*.png"),
                                                                ("All files", "*.*")))
        if self.image_path:
            image = Image.open(self.image_path)
            image = image.resize((400, 300), Image.LANCZOS)

            self.photo = ImageTk.PhotoImage(image)
            self.image_label.config(image=self.photo)

    def detect(self):
        if not self.image_path:
            self.output_label.config(text="Please load an image first.")
            return

        frame = cv2.imread(self.image_path)
        output_frame = integrate_models(frame)

        # Display processed image
        cv2.imshow('Processed Image', output_frame)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        # Save processed image
        output_image_path = 'processed_image.jpg'
        cv2.imwrite(output_image_path, output_frame)

        self.output_label.config(text="Detection completed. Processed image saved as processed_image.jpg.")

# Create GUI
window = tk.Tk()
app = DriverDetectionApp(window)
window.mainloop()
